In [2]:
import os
os.environ['PROJ_LIB'] = r'D:\\miniconda\\Lib\\site-packages\\osgeo\\data\\proj'

In [ ]:

import rasterio
from rasterio.warp import calculate_default_transform, reproject
import numpy as np
import geopandas as gpd


In [ ]:
def img_reproj(data_rio, ref_data=None, crs_dst=None):
    """
    重投影栅格数据函数。

    参数:
    data_rio: rasterio类对象，表示待重投影的栅格数据。
    ref_data: 参考数据，可以是栅格数据或矢量数据，提供重投影坐标系。
    crs_dst: epsg格式的坐标系，用于指定重投影的目标坐标系。

    返回:
    data_reproj_rio: 重投影后的rasterio栅格对象。
    """

    # 检查ref_data和crs_dst参数，确保至少有一个被设置
    if ref_data is None and crs_dst is None:
        raise ValueError("必须设置ref_data或crs_dst参数之一")

    # 如果ref_data是rasterio对象，获取其坐标系
    if isinstance(ref_data, rasterio.io.DatasetReader):
        crs_dst = ref_data.crs

    # 如果ref_data是geopandas的DataFrame，获取其坐标系
    elif isinstance(ref_data, gpd.GeoDataFrame):
        crs_dst = ref_data.crs.to_string()

    # 获取原始栅格数据的坐标系
    crs_src = data_rio.crs

    # 计算重投影所需的变换参数
    transform, width, height = calculate_default_transform(
        crs_src, crs_dst, data_rio.width, data_rio.height, *data_rio.bounds
    )

    # 创建一个新的rasterio对象，用于存储重投影后的数据
    kwargs = data_rio.meta.copy()
    kwargs.update({
        'crs': crs_dst,
        'transform': transform,
        'width': width,
        'height': height
    })

    # 重投影数据
    with rasterio.Env():
        with rasterio.open(data_rio.name) as src:
            data_reproj_rio = rasterio.MemoryFile()
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(data_reproj_rio, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=crs_dst,
                resampling=rasterio.warp.Resampling.bilinear
            )

    return data_reproj_rio

In [3]:
# 演示说明_1：通过crs_dst指定重投影坐标系
# 1. 读取原始栅格数据
data_rio = rasterio.open('E:\开源\cg_20.tif')

In [4]:
# 2. 设置目标坐标系（例如：EPSG:4326）
crs_dst = 'EPSG:4326'

# 3. 调用重投影函数
data_reproj_rio = img_reproj(data_rio, crs_dst=crs_dst)

# 4. 查看重投影后的数据
data_reproj_rio.plot()

CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: D:\miniconda\Library\share\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.

In [ ]:
# 演示说明_2:以ref_data为参考数据，提供重投影坐标系（接受栅格数据）
# 1. 读取原始栅格数据
data_rio = rasterio.open('E:\开源\chenggong_tif_wgs.tif')

# 2. 读取参考栅格数据
ref_raster_data = rasterio.open('path_to_your_ref_raster_file.tif')

# 3. 使用参考栅格数据进行重投影
data_reproj_rio_raster = img_reproj(data_rio, ref_data=ref_raster_data)

# 4. 查看重投影后的数据
data_reproj_rio_raster.plot()

In [ ]:
# 演示说明_3:以ref_data为参考数据，提供重投影坐标系（接受矢量数据）
# 1. 读取原始栅格数据
data_rio = rasterio.open('E:\开源\chenggong_tif_utm.tif')

# 2. 读取参考矢量数据
ref_vector_data = gpd.read_file('E:\开源\chenggong_wgs.shp')

# 3. 使用参考矢量数据进行重投影
data_reproj_rio_vector = img_reproj(data_rio, ref_data=ref_vector_data)

# 4. 查看重投影后的数据
data_reproj_rio_vector.plot()

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)